Version 1.0.0

# Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise. 

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.

Now go through the notebook and complete the assignment.

In [3]:
import numpy as np
import pandas as pd 
import scipy.sparse

# Load the data

Let's load the test data. Note, that we don't have any training data here, just test data. Moreover, *we will not even use any features* of test objects. All we need to solve this task is the file with the indices for the pairs, that we need to compare.

Let's load the data with test indices.

In [4]:
test = pd.read_csv('../readonly/data_leakages_data/test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


In [5]:
test.describe()

,pairId,FirstId,SecondId
count,368550.000000,368550.000000,368550.000000
mean,184274.500000,10863.601118,11950.398882
std,106391.365192,7280.190939,7602.814820
min,0.000000,0.000000,0.000000
25%,92137.250000,4574.000000,5698.000000
50%,184274.500000,9886.000000,10512.000000
75%,276411.750000,16512.000000,18782.000000
max,368549.000000,26324.000000,26324.000000


In [6]:
test.nunique()

pairId      368550
FirstId      26325
SecondId     26310
dtype: int64

In [7]:
grouped = test[['FirstId', 'SecondId']].groupby(['FirstId']).count().sort_values(by=['SecondId'], ascending=True)

In [8]:
pd.options.display.max_rows = 15
grouped

,SecondId
FirstId,
26324,7
24929,7
24931,7
24932,7
24933,7
24934,7
24935,7
...,...
1128,21


In [9]:
grouped.nunique()

SecondId    15
dtype: int64

In [10]:
test.sort_values(by=['FirstId', 'SecondId'])

,pairId,FirstId,SecondId
165695,165695,0,552
41153,41153,0,2299
328931,328931,0,3823
65725,65725,0,5959
289838,289838,0,6453
161023,161023,0,6546
297724,297724,0,7556
...,...,...,...
327667,327667,26324,3868
14755,14755,26324,4183


In [11]:
merged = test.merge(test, left_on=['FirstId', 'SecondId'], right_on=['SecondId', 'FirstId'])
merged = merged.sort_values(by=['FirstId_x', 'SecondId_x'])
merged

,pairId_x,FirstId_x,SecondId_x,pairId_y,FirstId_y,SecondId_y
23,33850,237,11620,203347,11620,237
153,293834,292,292,293834,292,292
3,2628,292,2587,144790,2587,292
41,86301,292,4115,365862,4115,292
194,353913,292,6251,28259,6251,292
122,239543,292,8657,276692,8657,292
134,258647,292,9968,101559,9968,292
...,...,...,...,...,...,...
90,181883,23755,2799,281136,2799,23755
32,58517,24662,8046,200238,8046,24662


In [12]:
test.query('(FirstId == 237) or (FirstId == 11620)')

,pairId,FirstId,SecondId
1196,1196,237,11876
5718,5718,11620,20947
14092,14092,237,9913
33850,33850,237,11620
36873,36873,11620,4482
43069,43069,237,12014
76228,76228,237,25216
...,...,...,...
310810,310810,237,18295
340405,340405,237,10242


For example, we can think that there is a test dataset of images, and each image is assigned a unique `Id` from $0$ to $N-1$ (N -- is the number of images). In the dataframe from above `FirstId` and `SecondId` point to these `Id`'s and define pairs, that we should compare: e.g. do both images in the pair belong to the same class or not. So, for example for the first row: if images with `Id=1427` and `Id=8053` belong to the same class, we should predict $1$, and $0$ otherwise. 

But in our case we don't really care about the images, and how exactly we compare the images (as long as comparator is binary).  

**We suggest you to try to solve the puzzle yourself first.** You need to submit a `.csv` file with columns `pairId` and `Prediction` to the grader. The number of submissions allowed is made pretty huge to let you explore the data without worries. The returned score should be very close to $1$.

**If you do not want to think much** -- scroll down and follow the instructions below.

# EDA and leakage intuition

As we already know, the key to discover data leakages is careful EDA. So let's start our work with some basic data exploration and build an intuition about the leakage.

First, check, how many different `id`s are there: concatenate `FirstId` and `SecondId` and print the number of unique elements. Also print minimum and maximum value for that vector.

In [13]:
# YOUR CODE GOES HERE
ids = pd.concat([test['FirstId'], test['SecondId']])
num_ids = ids.nunique()
num_ids

26325

and then print how many pairs we need to classify (it is basically the number of rows in the test set)

In [14]:
# YOUR CODE GOES HERE
num_pairs = len(test)
num_pairs

368550

Now print, how many distinct pairs it would be possible to create out of all "images" in the dataset?   

In [15]:
# YOUR CODE GOES HERE
num_possible = num_ids * (num_ids - 1) / 2
num_possible

346489650.0

So the number of pairs we are given to classify is very very small compared to the total number of pairs. 

To exploit the leak we need to **assume (or prove)**, that the total number of positive pairs is small, compared to the total number of pairs. For example: think about an image dataset with $1000$ classes, $N$ images per class. Then if the task was to tell whether a pair of images belongs to the same class or not, we would have $1000\frac{N(N-1)}{2}$ positive pairs, while total number of pairs was $\frac{1000N(1000N - 1)}{2}$.

Another example: in [Quora competitition](https://www.kaggle.com/c/quora-question-pairs) the task was to classify whether a pair of qustions are duplicates of each other or not. Of course, total number of question pairs is very huge, while number of duplicates (positive pairs) is much much smaller.

Finally, let's get a fraction of pairs of class `1`. We just need to submit a constant prediction "all ones" and check the returned accuracy. Create a dataframe with columns `pairId` and `Prediction`, fill it and export it to `.csv` file. Then submit to grader and examine grader's output. 

In [16]:
# YOUR CODE GOES HERE
submission = test.assign(Prediction=1)[['pairId', 'Prediction']]
submission.to_csv('submission.csv', index=False)
submission.head()

,pairId,Prediction
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


So, we assumed the total number of pairs is much higher than the number of positive pairs, but it is not the case for the test set. It means that the test set is constructed not by sampling random pairs, but with a specific sampling algorithm. Pairs of class `1` are oversampled.

Now think, how we can exploit this fact? What is the leak here? If you get it now, you may try to get to the final answer yourself, othewise you can follow the instructions below.   

# Building a magic feature

In this section we will build a magic feature, that will solve the problem almost perfectly. The instructions will lead you to the correct solution, but please, try to explain the purpose of the steps we do to yourself -- it is very important.

## Incidence matrix

First, we need to build an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix). You can think of pairs `(FirstId, SecondId)` as of edges in an undirected graph. 

The incidence matrix is a matrix of size `(maxId + 1, maxId + 1)`, where each row (column) `i` corresponds `i-th` `Id`. In this matrix we put the value `1` to the position `[i, j]`, if and only if a pair `(i, j)` or `(j, i)` is present in  a given set of pais `(FirstId, SecondId)`. All the other elements in the incidence matrix are zeros.   

**Important!** The incidence matrices are typically very very sparse (small number of non-zero values). At the same time incidence matrices are usually huge in terms of total number of elements, and it is **impossible to store them in memory in dense format**. But due to their sparsity incidence matrices **can be easily represented as sparse matrices**. If you are not familiar with sparse matrices, please see [wiki](https://en.wikipedia.org/wiki/Sparse_matrix) and [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html). Please, use any of `scipy.sparse` constructors to build incidence matrix. 

For example, you can use this constructor: `scipy.sparse.coo_matrix((data, (i, j)))`. We highly recommend to learn to use different `scipy.sparse` constuctors, and matrices types, but if you feel you don't want to use them, you can always build this matrix with a simple `for` loop. You will need first to create a matrix using `scipy.sparse.coo_matrix((M, N), [dtype])` with an appropriate shape `(M, N)` and then iterate through `(FirstId, SecondId)` pairs and fill corresponding elements in matrix with ones. 

**Note**, that the matrix should be symmetric and consist only of zeros and ones. It is a way to check yourself.

In [17]:
max_id = ids.max()
max_id

26324

In [18]:
num_ids - max_id

1

In [19]:
edges = test[['FirstId', 'SecondId']].drop_duplicates()
#entries = pd.concat([edges[['FirstId', 'SecondId']], edges[['SecondId', 'FirstId']]], axis = 0, ignore_index=True)
entries = pd.concat([edges[['FirstId', 'SecondId']], edges.rename(index=str, columns = {'FirstId':'SecondId', 'SecondId':'FirstId'})], axis = 0, ignore_index=True)
entries = entries.drop_duplicates(['FirstId', 'SecondId'], keep='first').reset_index(drop=True)
print(len(test))
print(len(edges))
print(len(entries))
entries.head()

368550
368538
736872


/home/mortennp/anaconda3/envs/AdvML3_6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,FirstId,SecondId
0,1427,8053
1,17044,7681
2,19237,20966
3,8005,20765
4,16837,599


In [20]:
entries[entries.duplicated()]

,FirstId,SecondId


In [21]:
entries.query('FirstId == SecondId')

,FirstId,SecondId
186439,300,300
218923,2551,2551
221185,8046,8046
228778,2799,2799
293825,292,292
327320,10404,10404


In [22]:
len(entries) - 736872

0

In [23]:
entries.query('FirstId == SecondId')

,FirstId,SecondId
186439,300,300
218923,2551,2551
221185,8046,8046
228778,2799,2799
293825,292,292
327320,10404,10404


In [24]:
# YOUR CODE GOES HERE (but probably you will need to write few more lines before)
inc_mat = scipy.sparse.coo_matrix(
    (np.ones(len(entries)), (entries['FirstId'], entries['SecondId'])),
    shape=(num_ids, num_ids),
    dtype=np.int
) 
#for i in range(num_ids):
#    if (2 == inc_mat[i,i]):
#        inc_mat[i,i] = 1
        
# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872

It is convenient to have matrix in `csr` format eventually.

In [25]:
inc_mat = inc_mat.tocsr()

## Now build the magic feature

Why did we build the incidence matrix? We can think of the rows in this matix as of representations for the objects. `i-th` row is a representation for an object with `Id = i`. Then, to measure similarity between two objects we can measure similarity between their representations. And we will see, that such representations are very good.

Now select the rows from the incidence matrix, that correspond to `test.FirstId`'s, and `test.SecondId`'s.

In [26]:
# Note, scipy goes crazy if a matrix is indexed with pandas' series. 
# So do not forget to convert `pd.series` to `np.array`
# These lines should normally run very quickly 

rows_FirstId   = inc_mat[test['FirstId'].values] # YOUR CODE GOES HERE
rows_SecondId  = inc_mat[test['SecondId'].values] # YOUR CODE GOES HERE

Our magic feature will be the *dot product* between representations of a pair of objects. Dot product can be regarded as similarity measure -- for our non-negative representations the dot product is close to 0 when the representations are different, and is huge, when representations are similar. 

Now compute dot product between corresponding rows in `rows_FirstId` and `rows_SecondId` matrices.

In [27]:
rows_FirstId

<368550x26325 sparse matrix of type '<class 'numpy.int64'>'
	with 11045662 stored elements in Compressed Sparse Row format>

In [28]:
# Note, that in order to do pointwise multiplication in scipy.sparse you need to use function `multiply`
# regular `*` corresponds to matrix-matrix multiplication

f = rows_FirstId.multiply(rows_SecondId).sum(axis=1) # YOUR CODE *GOES HERE
print(type(f))

<class 'numpy.matrixlib.defmatrix.matrix'>


In [29]:
f = np.array(f)
print(f.shape)
f=f.squeeze()
print(f.shape)

# Sanity check
assert f.shape == (368550, )

(368550, 1)
(368550,)


That is it! **We've built our magic feature.** 

# From magic feature to binary predictions

But how do we convert this feature into binary predictions? We do not have a train set to learn a model, but we have a piece of information about test set: the baseline accuracy score that you got, when submitting constant. And we also have a very strong considerations about the data generative process, so probably we will be fine even without a training set. 

We may try to choose a thresold, and set the predictions to 1, if the feature value `f` is higer than the threshold, and 0 otherwise. What threshold would you choose? 

How do we find a right threshold? Let's first examine this feature: print frequencies (or counts) of each value in the feature `f`.

In [30]:
# For example use `np.unique` function, check for flags
u = np.unique(f, return_counts=True)

u # YOUR CODE GOES HERE

(array([14, 15, 19, 20, 21, 28, 35]),
 array([183279,    852,    546, 183799,      6,     54,     14]))

Do you see how this feature clusters the pairs? Maybe you can guess a good threshold by looking at the values? 

In fact, in other situations it can be not that obvious, but in general to pick a threshold you only need to remember the score of your baseline submission and use this information. Do you understand why and how?  

Choose a threshold below: 

In [31]:
pred = f > 19 # SET THRESHOLD HERE

# Finally, let's create a submission

In [32]:
submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

submission.to_csv('submission3.csv', index=False)

In [33]:
submission.describe()

,pairId,Prediction
count,368550.000000,368550.000000
mean,184274.500000,0.498909
std,106391.365192,0.499999
min,0.000000,0.000000
25%,92137.250000,0.000000
50%,184274.500000,0.000000
75%,276411.750000,1.000000
max,368549.000000,1.000000


Now submit it to the grader! It is not possible to submit directly from this notebook, as we need to submit a `csv` file, not a single number (limitation of Coursera platform). 

To download `submission.csv` file that you've just produced <a href='./submission.csv'>click here</a> (if the link opens in browser, right-click on it and shoose "Save link as"). Then go to [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.


If you did everything right, the score should be very high.

**Finally:** try to explain to yourself, why the whole thing worked out. In fact, there is no magic in this feature, and the idea to use rows in the incidence matrix can be intuitively justified.

# Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!